In [18]:
import json
import os
import numpy as np
import math
from typing import Dict, List, Tuple
import copy
from itertools import combinations
from collections import Counter

import logging
from datetime import timedelta
from config import *
from utils import *
from main import *
from GtConflictFinder import GtConflictFinder
from ConflictDetector import ConflictDetector, ConflictPredicator


import plotly.graph_objects as go
from plotly.subplots import make_subplots

test_projects = [
    # "House1",
    "House3",
    "House4",
    "House8",
    "House9",
    "House15",
    # "House18",
    # "House20",
]
ctx_info = ContextAccessor({
            TIME_CTX: {
                "range" : (0, 24*60),
                "interval" : 60,
            },
            # "humidity#NUM" : {
            #     "range" : (0, 100),
            #     "interval" : 10,
            # },
            # "FeelsLikeC#NUM" : {
            #     "range" : (-10., 40),
            #     "interval" : 5,
            # },
            # "tempC#NUM" : {
            #     "range" : (-10., 40),
            #     "interval" : 5,
            # },
            "weatherDesc#CAT": {},
            WEEKDAY_CTX: {
                "range": (0, 6),
                "interval": 1,
            },
            HOLIDAY_CTX: {
                "range": (0,1),
                "interval": 1,
            }
        })
capacity = {
    "TV":1,
    "WashingMachine":1,
    "PC": 1,
}
BOOL_SIM = False
BOOL_UMASS= False
ccp_alpha = 2e-4
# test_dates = generate_test_date(test_projects, test_ratio = 0, true_random=False, is_sim=BOOL_SIM, is_umass=BOOL_UMASS)
test_dates = []


In [19]:
habit_groups = {}
for p in test_projects:
    habit_groups[p], grid_data = test_umass(test_project=p, ctx_info=ctx_info, ccp_alpha=ccp_alpha, test_dates=test_dates, is_sim=BOOL_SIM, is_umass=BOOL_UMASS)

In [20]:
#Exps:
#   1. Contrived environment with injected conflicts:
#   2. Real dataset with groundtruth conflicts:
#   3. observations about the algorithm: with different threshold, different capacity for the devices, different number of users.

In [21]:
for home, groups in habit_groups.items():
    print("The habit groups found in {}".format(home))
    for d, d_groups in groups.items():
        print(d + " " + str(len(d_groups)))
        # for g in d_groups:
        #     print(ctx_info.coor_box_to_range(g["box"][0] + g["box"][1]))
        #     print(g["dis"])


The habit groups found in House3
WashingMachine 520
TV 653
The habit groups found in House4
PC 250
WashingMachine 170
TV 696
The habit groups found in House8
PC 403
WashingMachine 337
TV 447
The habit groups found in House9
WashingMachine 388
TV 458
The habit groups found in House15
TV 560
WashingMachine 488


In [22]:
c_detector = ConflictDetector(ctx_info, capacity)
final_conflicts = c_detector.predict_conflict_scenarios(habit_groups)
print({x:len(final_conflicts[x]) for x in final_conflicts})

KeyboardInterrupt: 

In [15]:
import plotly.express as px
# for x in final_conflicts["Window"]:
#     print((ctx_info.coor_box_to_range(x["box"]),x["prob"]))
probs_i = [(x["prob"], i) for i, x in enumerate(final_conflicts["TV"])]
# print(sorted(probs_i))
# fig = px.histogram(probs)
# fig.show()

In [7]:
import copy
# Make ground truth:

gt_ctx_info = ContextAccessor({
            TIME_CTX: {
                "range" : (0, 24*60),
                "interval" : 60,
            },
            # "humidity#NUM" : {
            #     "range" : (0, 100),
            #     "interval" : 10,
            # },
            # "FeelsLikeC#NUM" : {
            #     "range" : (-10., 40),
            #     "interval" : 5,
            # },
            # "tempC#NUM" : {
            #     "range" : (-10., 40),
            #     "interval" : 5,
            # },
            "weatherDesc#CAT": {},
            WEEKDAY_CTX: {
                "range": (0, 6),
                "interval": 1,
            },
            HOLIDAY_CTX: {
                "range": (0,1),
                "interval": 1,
            },
        })
device_events = {}
for p in test_projects:
    ctx_evts, device_evts = load_processed(p, is_sim=BOOL_SIM, is_umass=BOOL_UMASS)
    device_events[p] = device_evts

gtconflict_cfg = {
    "context_info": gt_ctx_info,
    "capacity": capacity
}
test_dates = {}

conflict_finder = GtConflictFinder(gtconflict_cfg)
gt_conflicts, test_state_cnt = conflict_finder.get_Gt_conflict(ctx_evts, device_events, test_dates)
print(len(gt_conflicts))

15152


In [8]:

# TEST_RATIO = 0.3
# conflict_time = [x['cur_time'] for x in conflicts]
# end_time = max(conflict_time)
# start_time = min(conflict_time)
# print(start_time)
# print(end_time)
# total_time_range = end_time - start_time
# test_start = end_time - total_time_range * TEST_RATIO
# print(test_start)

conflict_device = {
    d:[]
    for d in capacity
}

conflict_state_device = {
    d:{}
    for d in capacity
}
MIN_TEST_OBS = 20
cnts = []
for c in gt_conflicts:
    d = c["device"]
    conflict_device[d].append(c)
    s = gt_ctx_info.get_coor_by_ctx(c["ctx"])
    users = frozenset(c["device_states"].keys())
    cnts.append(test_state_cnt[d][s])
    if test_state_cnt[d][s] < MIN_TEST_OBS:
        continue
    if s not in conflict_state_device[d]:
        conflict_state_device[d][s] = {}
    conflict_state_device[d][s][users] = conflict_state_device[d][s].get(users, 0) + 1

    # if conflict_state_device[d][s][users] > test_state_cnt[d][s]:
    #     print(conflict_state_device[d][s][users], test_state_cnt[d][s])
    #     print(users, d, c["cur_time"], s)
    
print({d:len(c) for d,c in conflict_device.items()})
print({d:len(c) for d,c in conflict_state_device.items()})
print(len(test_dates))
print(Counter(cnts))

{'TV': 14841, 'WashingMachine': 299, 'PC': 12}
{'TV': 229, 'WashingMachine': 126, 'PC': 8}
0
Counter({41.0: 1435, 32.0: 1158, 37.0: 853, 29.0: 801, 54.0: 765, 24.0: 652, 28.0: 611, 40.0: 588, 44.0: 588, 51.0: 550, 45.0: 510, 47.0: 476, 23.0: 474, 25.0: 462, 4.0: 421, 3.0: 395, 21.0: 380, 1.0: 362, 36.0: 357, 5.0: 346, 15.0: 311, 17.0: 272, 2.0: 224, 50.0: 204, 19.0: 198, 8.0: 190, 12.0: 182, 20.0: 141, 7.0: 126, 22.0: 121, 14.0: 114, 16.0: 108, 39.0: 84, 9.0: 84, 43.0: 75, 42.0: 74, 57.0: 65, 6.0: 65, 55.0: 58, 13.0: 53, 35.0: 51, 60.0: 50, 18.0: 46, 38.0: 27, 46.0: 25, 26.0: 6, 33.0: 4, 10.0: 4, 59.0: 4, 31.0: 1, 34.0: 1})


In [ ]:
all_users = test_projects
all_devices = capacity.keys()
user_pairs = list(combinations(all_users, 2))
conflict_predicator = ConflictPredicator(ctx_info, final_conflicts)
exp_cnt = 0
exp_gt_c = 0
exp_result = {d:[0,0,0,0,0] for d in all_devices}
max_p = 0
all_state_cnt = 0
for d in all_devices:
    it = np.nditer(test_state_cnt[d], flags=['multi_index'])
    for count in it:
        all_state_cnt += count
        if count < MIN_TEST_OBS:
            continue
        state = it.multi_index
        for u_pair in user_pairs:
            gt_prob = 0.
            if (d in conflict_state_device) and \
                (state in conflict_state_device[d]) and \
                (frozenset(u_pair) in conflict_state_device[d][state]):

                gt_prob =  float(conflict_state_device[d][state][frozenset(u_pair)]) / count
            gt_ctx_snapshot = gt_ctx_info.coor_to_snapshot(state)

            pred_prob = conflict_predicator.get_prob_conflict(ctx_info.get_coor_by_ctx(gt_ctx_snapshot), u_pair, d)
            exp_cnt += 1
            if gt_prob > 0:
                # if gt_prob > 0.5:
                #     print(gt_ctx_info.coor_to_snapshot(state).values(), gt_prob, pred_prob, u_pair)
                exp_gt_c += 1
                exp_result[d][0] += abs(pred_prob - gt_prob)
                exp_result[d][1] += 1
                exp_result[d][4] += gt_prob
                if gt_prob > 1:
                    print(gt_prob, pred_prob, d, u_pair)
                max_p = max(gt_prob, max_p)

            else:
                exp_result[d][2] += pred_prob - gt_prob
                exp_result[d][3] += 1

for d in exp_result:
    # exp_result[d].append((exp_result[d][0] + exp_result[d][2]) / (exp_result[d][1] + exp_result[d][3]))
    if exp_result[d][1] > 0:
        exp_result[d][0] = exp_result[d][0] / exp_result[d][1]
        exp_result[d][4] = exp_result[d][4] / exp_result[d][1]
    if exp_result[d][3] > 0:
        exp_result[d][2] = exp_result[d][2] / exp_result[d][3]
    
print(max_p)
print(exp_result)
print(all_state_cnt)

0.72
{'TV': [0.06582888402017852, 1516, 0.008189849345400985, 1184, 0.22039600129188766], 'WashingMachine': [0.03225944612887707, 178, 0.000908910773986847, 2522, 0.0368411225925334], 'PC': [0.03007722666261308, 8, 3.4600824185895246e-05, 2992, 0.03240663719146748]}
41907.0
